#import


In [0]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.nn.functional as F
import time as t

def get_dataset(resol_max,path, validation = False, database = 'cifar10', download = True):

    transform = transforms.Compose([transforms.Resize(size=(resol_max, resol_max)), transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])
    if validation:
        raise NotImplementedError
    if database=='cifar10':
        trainset = torchvision.datasets.CIFAR10(root= path, train=True, transform=transform, download=download)
        testset = torchvision.datasets.CIFAR10(root = path, train=False,transform=transform, download=download)
    else:
        raise NotImplementedError
    return trainset, testset


def get_loader(data, batch_size, num_workers=2):
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True,num_workers = num_workers)
    return loader


def accuracy(net,testset,batch_size = 64,cuda=True):
    r=net.resolution
    testloader = get_loader(testset, batch_size)
    net.eval()
    correct = 0
    total = 0
    if cuda:
        net.to('cuda')
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            if cuda:
                images = images.to('cuda')
                labels = labels.to('cuda')
            ## dans tous les cas ici il faut changer la resol de l'image 
            images = F.interpolate(images, r)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
           
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        return 100.0 * correct/total




def train(model, optimizer, trainset, loss_fn, valset=None, batch_size=64,  n_epoch = 5, cuda=True, disp_stats = True, validate = False):
    if cuda:
        model.to('cuda')

    if validate :
        valloader = get_loader(valset, batch_size)


    resol_cible = model.resolution
    trainloader = get_loader(trainset, batch_size)

    loss_train, loss_val = [], []
    acc_train, acc_val = [],[]

    loss_func = loss_fn()
    
    for epoch in range(n_epoch):
        running_trainloss = 0
        for data in trainloader:
            inputs, labels = data
            if cuda:
                inputs = inputs.to('cuda') 
                labels = labels.to('cuda')

            # on change la resol de l'image    
            optimizer.zero_grad()
            inputs = F.interpolate(inputs, resol_cible)
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            running_trainloss+= loss
            optimizer.step()
    
        # maintenant plus qu'a calculer les stats et les afficher
        loss_train.append(running_trainloss)
        acct = accuracy(model, trainset, batch_size=batch_size)
        acc_train.append(acct)
        if disp_stats:
            print('Epoch', epoch, '/', n_epoch, ':')
            print('train loss : ', running_trainloss)
            print('train acc',acct )

        # stats de validation
        if validate:
            vloss = 0
            for data in valloader:
                ipt, lbl = data
                if cuda : 
                    ipt = ipt.to('cuda')
                    lbl = ipt.to('cuda')
                ipt = F.interpolate(ipt, resol_cible)
                out = model(ipt)
                vloss+= loss_func(out, lbl)
            loss_val.append(vloss)
            accv = accuracy(model, valset, batch_size)
            acc_val.append(accv)
            if disp_stats:
                print('val loss : ', vloss)
                print('val acc' ,accv)

    if validate:
        raise NotImplementedError
    else:
        return model, loss_train, acc_train




class ResNetBlock(nn.Module):
  def __init__(self, in_planes, planes, stride=1):
      super(ResNetBlock, self).__init__()
      self.stride = stride
      self.in_planes=in_planes
      self.planes = planes
      if stride!=1:

        self.fx = nn.Sequential(nn.Conv2d(in_planes, planes, 3, stride=2, 
                                          padding=1),
                                nn.ReLU(), 
                                nn.Conv2d(planes, planes,3, padding=1))
        
        self.iden = nn.Conv2d(self.in_planes, self.planes, 3, stride = 2,
                              padding =1)

      else:
        self.fx = nn.Sequential(nn.Conv2d(planes, planes, 3, padding = 1),
                                nn.ReLU(), 
                                nn.Conv2d(planes, planes,3, padding=1))
        self.iden = nn.Sequential()
          

  def forward(self, x):

    if self.stride ==1:
      fx = self.fx(x)
      if fx.size()!=self.iden(x).size():
        print(fx.size(),self.iden(x).size() )
        print("1")
      
      out = fx + self.iden(x)
      return F.relu(out)

    else:

      fx = self.fx(x)
      if fx.size()!=self.iden(x).size():
        print(fx.size(),self.iden(x).size() )
        print("2")
    
      out = fx + self.iden(x)
      return F.relu(out)




class ResNet(nn.Module):

  def __init__(self, depth,resolution,width=16, num_classes=10,input_dim=3,Block = ResNetBlock):
      super(ResNet, self).__init__()
      self.in_planes = width
      self.conv1 = nn.Conv2d(input_dim, width, kernel_size=3, stride=1, padding=1, bias=False)
      self.bn1 = nn.BatchNorm2d(width)
      self.depth = depth
      self.width = width
      self.resolution = resolution
      
      resol = resolution
      # construction des piles
      plane = width
      layers = []
      for nb in depth:
        layer = self._make_layer(Block,plane ,nb,2)
        layers.append(layer)
        plane*=2
        resol/=2
      
      self.layers = nn.Sequential(*layers)
      # arrivé ici on en est à plane@resol*resol
      # on applique le pooling
      self.pool = nn.AvgPool2d(4)
      resol = int(resol/4)

      out_size = resol*resol*plane 
      self.linear = nn.Linear(out_size, num_classes)

  def _make_layer(self, Block, planes, num_blocks, stride):
      layers = []
      block1 = Block(planes, 2*planes, stride = 2)
      planes *=2
      layers.append(block1)
      for i in range(1,num_blocks):
        block_new = Block(planes, planes, stride =1)
        layers.append(block_new)
      return nn.Sequential(*layers)

  def forward(self, x):
      
    # premiere etape
      out = F.relu(self.bn1(self.conv1(x)))
     
      # passage par les piles 
      out = self.layers(out)
      
      # output
      out = self.pool(out)
      out = out.view(out.size(0), -1)
      out = self.linear(out)
      return out

def get_resnet(depth = [2,2,2,2],resolution = 224,width=16, num_classes=10,input_dim=3):
  resnet = ResNet(depth,resolution,width,num_classes=10,input_dim=3)
  return resnet

def get_new_depth(alpha, depth):
  old_sum_depth = sum(depth)
  new_sum_depth = int(alpha*old_sum_depth)
  new_depth = depth 
  new_sum = old_sum_depth
  for i in range(len(depth)):
    if new_sum < new_sum_depth:
      new_depth[i]+=1
      new_sum +=1
  return new_depth


def gridsearch(trainset, testset,loss_fn,precision=.2,epsilon = .5,phi=1, bs=64,learning_rate=1e-2,n_epoch=2):
  solu=1,1,1
  depth, width, resol = [2,2,2,2], 16, 112  ## ce sont les valeurs de base à scaler ensuite
  # training
  print('entrainement modele 1')
  model1 = get_resnet(depth, resol, width)
  optimizer = torch.optim.Adam(model1.parameters(),lr=learning_rate)
  debut = t.time()
  model1, _, _ =train(model1, optimizer, trainset, loss_fn, batch_size=bs, n_epoch=n_epoch, disp_stats=False, validate=False)
  fin = t.time()
  print('temps d entrainement : ', fin-debut)
  # testing
  accu_max=accuracy(model1, testset, batch_size=bs)

  for alpha in np.arange(1,2,precision):
    for beta in np.arange(1,2,precision):
      for gamma in np.arange(1,2,precision):
        if np.abs(alpha*beta**2*gamma**2 - 2)<=epsilon:
          # il faut réentraîner le modele
          print('entrainement avec parametres', alpha, beta, gamma)
          new_depth = get_new_depth(alpha, depth)
          
          print("width : ", int(beta*width))
          print("resol : ", int(gamma*resol))
          print("depth: ",  new_depth)
          model = get_resnet(depth = new_depth, width = int(beta*width), resolution = int(gamma*resol))

          optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
          deb = t.time()
          model, _, _ = train(model, optimizer, trainset, loss_fn, batch_size=bs, n_epoch=n_epoch, disp_stats=False, validate=False)
          fin = t.time()
          print("temps d entrainement : ", fin-deb)
          accu = accuracy(model, testset, batch_size = bs)
          if accu>accu_max:
            accu_max= accu
            solu=alpha,beta,gamma
    return solu


# tests

In [2]:
trainset, testset = get_dataset(224, '/')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
loss_fn = torch.nn.CrossEntropyLoss


In [0]:
# pour l'entrainement c'est environ 4 min par epoch

In [13]:
gridsearch(trainset, testset, loss_fn, n_epoch = 1, bs=220)

entrainement modele 1
temps d entrainement :  200.72062635421753
entrainement avec parametres 1.0 1.0 1.4
width :  16
resol :  156
depth:  [2, 2, 2, 2]
temps d entrainement :  437.00736904144287
entrainement avec parametres 1.0 1.2 1.2
width :  19
resol :  134
depth:  [2, 2, 2, 2]
temps d entrainement :  674.967503786087
entrainement avec parametres 1.0 1.4 1.0
width :  22
resol :  112
depth:  [2, 2, 2, 2]
temps d entrainement :  909.3671100139618


(1, 1, 1)